In [ ]:
%pip install -r requirements.txt

In [ ]:
# define training data for our simple ai ops model

bad_texts = [
    "service is down in production",
    "database connection refused",
    "error 500 on checkout",
    "latency is spiking above 2s",
    "memory leak detected in worker",
    "disk is almost full on node 3",
    "cpu pegged at 100 percent",
    "timeout talking to upstream api",
    "critical alert: payment failures",
    "users cannot log in",
    "incident declared sev1",
    "pager going off nonstop",
    "major outage across regions",
    "kubernetes pod crashloopbackoff",
    "ssl certificate expired",
    "data corruption detected"
]

good_texts = [
    "deploy completed successfully",
    "all systems operational",
    "health check passing",
    "no issues detected",
    "latency back to normal",
    "service recovered after restart",
    "incident resolved",
    "error rate returned to baseline",
    "backup completed successfully",
    "scaling up completed",
    "cache warmed successfully",
    "migration finished without errors",
    "api responding normally",
    "monitoring is green",
    "rollout finished",
    "tests passed in ci"
]

train_texts = bad_texts + good_texts
train_labels = [1]*len(bad_texts) + [0]*len(good_texts)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(train_texts, train_labels)

In [ ]:
# optional, test the model inline

samples = [
    "checkout is failing with 500 errors",
    "deploy finished and everything looks good",
    "pods are crashlooping in prod",
    "all systems operational"
]

pred = model.predict(samples)
proba = model.predict_proba(samples)

for t, y, p in zip(samples, pred, proba):
    print(y, round(float(p[1]), 3), t)

In [ ]:
# save the model as bentoml package
import bentoml
model_ref = bentoml.sklearn.save_model(
    "ops_sentiment",
    model,
    signatures={"predict": {"batchable": True}, "predict_proba": {"batchable": True}},
)



In [12]:
# optional run in uvicorn
!python -m uvicorn service:app --host 127.0.0.1 --port 7777 --reload

^C


In [ ]:
# !python -m bentoml build
# optional list containers
# !python -m bentoml bentos list
# !python -m bentoml containerize ops_sentiment_service:latest
